In [66]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

In [67]:
import pandas as pd

# Load the data
regular_season = pd.read_csv("Consolidated Regular Season Data.csv")
playoffs = pd.read_csv("Consolidated Playoffs - Sheet1.csv")

# Remove asterisks from the team names in the regular season data
regular_season['Team'] = regular_season['Team'].str.replace('*', '', regex=False)

# Ensure data types are consistent for merging
playoffs['Year'] = playoffs['Year'].astype(int)
regular_season['Season (end)'] = regular_season['Season (end)'].astype(int)

In [68]:
# Merge regular season data with playoffs data for losers
merged = pd.merge(regular_season, playoffs, left_on=['Season (end)', 'Team'], right_on=['Year', 'Loser'], how='left')
merged['Lost To'] = merged['Winner']
merged['Year'] = merged['Season (end)']
merged['Round made it to'] = merged['Playoff Round']
merged.drop(columns=['Season (end)', 'Playoff Round', 'Winner', 'Loser'], inplace=True)

merged

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,TOV%.1,DRB%,FT/FGA.1,Successful,Arena,Attend.,Attend./G,Year,Lost To,Round made it to
0,1,Miami Heat,29.0,58,24,61,21,7.46,-0.71,6.76,...,12.6,75.5,0.223,1,AmericanAirlines Arena,"810,930","19,779",2011,Dallas Mavericks,Finals
1,2,Chicago Bulls,26.4,62,20,61,21,7.32,-0.79,6.53,...,13.6,76.2,0.222,1,United Center,"893,462","21,792",2011,Miami Heat,Eastern Conference Finals
2,3,Los Angeles Lakers,30.2,57,25,58,24,6.11,-0.10,6.01,...,12.8,72.3,0.181,0,STAPLES Center,"778,877","18,997",2011,Dallas Mavericks,Western Conference Semifinals
3,4,San Antonio Spurs,28.8,61,21,56,26,5.71,0.16,5.86,...,12.6,74.6,0.190,0,AT&T Center,"750,879","18,314",2011,Memphis Grizzlies,Western Conference First Round
4,5,Boston Celtics,29.5,56,26,57,25,5.37,-0.54,4.83,...,14.8,74.7,0.235,0,TD Garden,"763,584","18,624",2011,Miami Heat,Eastern Conference Semifinals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,26,Portland Trail Blazers,25.1,33,49,31,51,-4.01,0.05,-3.96,...,12.1,74.9,0.217,0,Moda Center,"767,374","18,716",2023,NaN,NaN
386,27,Charlotte Hornets,25.3,27,55,26,56,-6.24,0.35,-5.89,...,12.5,75.5,0.211,0,Spectrum Center,"702,052","17,123",2023,NaN,NaN
387,28,Houston Rockets,22.1,22,60,23,59,-7.85,0.24,-7.62,...,11.8,75.8,0.218,0,Toyota Center,"668,865","16,314",2023,NaN,NaN
388,29,Detroit Pistons,24.1,17,65,22,60,-8.22,0.49,-7.73,...,11.9,74.0,0.231,0,Little Caesars Arena,"759,715","18,596",2023,NaN,NaN


In [69]:
# For winners, especially for identifying NBA Champions
winners = playoffs[playoffs['Playoff Round'] == 'Finals'][['Winner', 'Year']]
winners['Round made it to'] = 'NBA Champion'
winners.rename(columns={'Winner': 'Team'}, inplace=True)

winners

,Team,Year,Round made it to
0,Denver Nuggets,2023,NBA Champion
15,Golden State Warriors,2022,NBA Champion
30,Milwaukee Bucks,2021,NBA Champion
45,Los Angeles Lakers,2020,NBA Champion
60,Toronto Raptors,2019,NBA Champion
75,Golden State Warriors,2018,NBA Champion
90,Golden State Warriors,2017,NBA Champion
105,Cleveland Cavaliers,2016,NBA Champion
120,Golden State Warriors,2015,NBA Champion
135,San Antonio Spurs,2014,NBA Champion


In [71]:
merged.set_index(['Team', 'Year'], inplace=True)
winners.set_index(['Team', 'Year'], inplace=True)

# Update 'Round made it to' for matching index entries in 'merged' using 'winners'
merged.loc[merged.index.isin(winners.index), 'Round made it to'] = 'NBA Champion'

# Reset the index to revert back to the standard dataframe format
merged.reset_index(inplace=True)

# Now, the 'merged' dataframe will reflect 'NBA Champion' in the 'Round made it to' column for the winners
merged.to_csv('NBA_Total_Season_Data.csv', index=False)

KeyError: "None of ['Team', 'Year'] are in the columns"